In [ ]:
#로컬에 설치한 AgensGraph를 python으로 접근하여 테이블 데이터 생성하기기
import psycopg2
from faker import Faker
import random
from datetime import timedelta

# PostgreSQL 연결 정보
conn = psycopg2.connect(
    dbname="agens",
    user="postgres",
    password="agens",
    host="localhost",
    port="5455"
)

cur = conn.cursor()
faker = Faker()

# 가상 데이터 생성
transaction_types = ['deposit', 'withdrawal', 'transfer']

for _ in range(1000):
    account_id = random.randint(1, 100)  # 예시: 계좌 ID가 1~100 사이 있다고 가정
    transaction_type = random.choice(transaction_types)
    amount = round(random.uniform(10.00, 10000.00), 2)
    transaction_date = faker.date_time_between(start_date='-1y', end_date='now')
    description = faker.sentence(nb_words=6)

    cur.execute("""
        INSERT INTO transactions (account_id, transaction_type, amount, transaction_date, description)
        VALUES (%s, %s, %s, %s, %s)
    """, (account_id, transaction_type, amount, transaction_date, description))

# 커밋 및 종료
conn.commit()
cur.close()
conn.close()

print("1000개의 거래 데이터가 삽입되었습니다.")

In [ ]:
# 랭체인과 오픈AI설치 안되어 있으면 설치할것것
pip install langchain
pip install langchain_openai

In [35]:
# 랭채인에 있는 오픈AI 라이브러리리
from langchain_openai import ChatOpenAI

# 랭채인 필수 기본 라이브러리
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate

In [3]:
# 랭체인의 기본 질의 응답
llm = ChatOpenAI(model="gpt-4o-mini")
answer = llm.invoke('OECD국가는 몇개이며 한국은 경제규 몇위인지?')
answer

AIMessage(content='OECD(경제협력개발기구)는 38개의 회원국으로 구성되어 있습니다. 한국은 OECD 회원국 중 하나이며, 경제규모 면에서는 대체로 10위권 내에 위치해 있습니다. 구체적인 순위는 매년 경제 성장률 및 통계에 따라 변동이 있을 수 있으므로, 최신 자료를 통해 확인하는 것이 좋습니다. 예를 들어, 2023년 기준으로 한국의 경제규모는 대략 10위로 평가되고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 22, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-Bbdv87hdXLI2lXhjOTxZW3AvE3H0U', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e0f4dab-ae5b-4010-9168-ae9c1e452688-0', usage_metadata={'input_tokens': 22, 'output_tokens': 108, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

In [ ]:
prompt = ChatPromptTemplate.from_template("너는 천문학 분야 전문가로써 질문에 답변을 해줘 <Question>:{input}")

chain = prompt | llm
chain.invoke({"input" : "지구의 자전 주기는?"})

AIMessage(content="지구의 자전 주기는 약 24시간입니다. 정확히 말하면, 지구는 한 번 자전하는 데 약 23시간 56분 4초가 걸립니다. 이를 '항성일'이라고 하며, 일반적으로 우리가 사용하는 24시간은 태양이 하늘에서 이동하는 것을 기준으로 하는 '태양일'을 나타냅니다. 태양일은 약간 길어지는 이유는 지구가 태양 주위를 공전하면서 회전하기 때문입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 34, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BZseqb7LffJ2JT2lJigKXwVEZ9sMZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3eb2ba6-aa59-49c5-886e-fb8d0f95c676-0', usage_metadata={'input_tokens': 34, 'output_tokens': 108, 'total_tokens': 142, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke({"input" : "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 좀 더 정확히 말하면, 평균적으로 23시간 56분 4초 정도입니다. 이 시간을 “자전일”이라고 하며, 이는 지구가 한 번 자전하는 데 걸리는 시간입니다. 그러나 우리가 일반적으로 사용하는 24시간은 평균적인 태양일을 기준으로 하며, 태양이 다시 같은 위치에 나타나는 시간을 기준으로 합니다.'

In [13]:
prompt1 = ChatPromptTemplate.from_template("translates {Korean_word} to English and one word answer")
prompt2 = ChatPromptTemplate.from_template("explain {English_word} using oxford dictionary to me in korean ")

llm = ChatOpenAI(model = 'gpt-4o-mini')
chain1 = prompt1 | llm | StrOutputParser()
chain1.invoke({"Korean_word" : "미래"})

'Future'

In [14]:
chain2 = (
    {"English_word" : chain1} | prompt2 | llm | StrOutputParser()
          )

chain2.invoke({"Korean_word" : "미래"})

'"Future"라는 단어는 옥스포드 사전에서 다음과 같이 정의됩니다:\n\n1. **미래**: 시간이 아직 오지 않은 시점, 즉 현재와 과거 사이에 위치한 시간.\n2. **미래의 일들**: 앞으로 발생할 가능성이 있는 사건이나 상황들.\n\n한국어로 설명하자면, \'미래\'는 현재와 과거 사이에 있으며, 앞으로 일어날 가능성이 있는 상황이나 사건들을 의미합니다. 예를 들어, 우리의 계획이나 꿈, 예상되는 변화 등이 모두 미래와 관련이 있습니다.'

In [7]:
# langchain의 기존 사용 구조

# 1. 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("전문가의 관점으로 {input}을 쉽고 간단히 설명해줘")
model = init_chat_model("gpt-4o-mini", model_provider="openai")
output_parser = StrOutputParser()

# 2. 체인생성
chain = prompt | model | output_parser

# 3. 메소드 사용
result = chain.invoke({"input":"langchain공부법"})
print(result)

LangChain은 자연어 처리(NLP)와 관련된 다양한 작업을 수행할 수 있게 해주는 프레임워크입니다. LangChain을 효과적으로 공부하기 위한 방법을 단계별로 설명하겠습니다.

### 1. 기초 개념 이해
- **자연어 처리(NLP)**: 언어 모델, 텍스트 처리, 의미 분석 등의 기초 개념을 이해합니다.
- **파이썬 언어**: LangChain은 주로 파이썬으로 작성되므로, 파이썬의 기본 문법과 라이브러리 사용에 익숙해져야 합니다.

### 2. 공식 문서와 튜토리얼 활용
- **공식 문서**: LangChain의 [공식 문서](https://langchain.readthedocs.io/en/latest/)를 읽으며 기본적인 사용법과 API를 익힙니다.
- **튜토리얼**: 다양한 튜토리얼을 통해 실습하면서 개념을 확립합니다. Code snippets를 통해 실제 코드 예제를 따라 해보는 것이 도움이 됩니다.

### 3. 실습 프로젝트
- **작은 프로젝트 시작**: 간단한 챗봇이나 텍스트 요약기 같은 작은 프로젝트를 진행하여 실습합니다. 이 과정에서 LangChain의 다양한 기능을 체험할 수 있습니다.
- **오픈소스 프로젝트 참여**: GitHub 등에서 LangChain 관련 오픈소스 프로젝트를 찾아 기여하거나 참고합니다.

### 4. 커뮤니티와의 소통
- **포럼이나 Slack 채널**: LangChain 관련 커뮤니티에 참여하여 다른 사용자들과 소통하고 질문합니다.
- **미팅 및 웨비나**: 관련 세미나나 워크숍에 참석하여 최신 정보와 네트워킹 기회를 얻습니다.

### 5. 심화 학습
- **고급 기능 익히기**: LangChain의 고급 기능이나 모듈(예: LangChain Agents, Chains 등)을 심화 학습합니다.
- **기타 라이브러리와의 연계**: LangChain과 함께 사용할 수 있는 다른 라이브러리(예: Hugging Face의 Transformers, OpenAI API 등)에 대한 연구와 실습을 합니다.



In [20]:
# 4-1. 메서드사용
inputs = ["LangGraph","LangSmith","LangChain"]
results = chain.batch([{"input":i} for i in inputs])
results

['LangGraph은 언어와 관련된 다양한 데이터를 시각적으로 나타내고 분석할 수 있는 도구입니다. 더 쉽게 설명하자면, 텍스트와 단어의 관계를 그래프로 표현하여 언어의 구조와 의미를 이해하는 데 도움을 주는 플랫폼입니다.\n\n1. **언어 데이터 시각화**: LangGraph은 단어, 문장, 또는 문서 간의 관계를 그래프 형태로 나타냅니다. 이를 통해 사용자는 특정 단어가 다른 단어와 어떻게 연결되어 있는지 쉽게 파악할 수 있습니다.\n\n2. **관계 분석**: 예를 들어, 단어의 동의어, 대칭어, 그리고 문맥에 따라 어떻게 쓰이는지를 분석하여 언어의 뉘앙스를 이해하게 합니다.\n\n3. **사용자 친화적인 인터페이스**: 복잡한 NLP(자연어 처리) 기술 없이도 그래프를 통해 데이터를 탐색하고 언어의 패턴을 발견할 수 있는 직관적인 인터페이스를 제공합니다.\n\n4. **교육 및 연구 도구**: 언어학, 문학 연구, 교육 등의 분야에서 유용하게 사용할 수 있습니다. 학습자가 언어 구조를 이해하는 데 큰 도움이 됩니다.\n\n이렇게 LangGraph은 언어의 패턴을 시각적으로 쉽게 이해하고, 활용할 수 있도록 돕는 도구입니다.',
 'LangSmith는 주로 자연어 처리(NLP)와 관련된 도구 또는 플랫폼으로, 사용자들이 다양한 언어 모델을 쉽게 구축하고 관리할 수 있도록 돕는 솔루션입니다. 이 플랫폼은 개발자나 데이터 과학자가 복잡한 알고리즘이나 수학적 모델링 없이도 언어 모델을 실험하고 적용할 수 있는 환경을 제공합니다.\n\nLangSmith의 주요 기능은 다음과 같습니다:\n\n1. **모델 훈련**: 사용자들이 데이터를 입력하여 자신만의 언어 모델을 훈련시킬 수 있도록 지원합니다.\n\n2. **프로토타이핑**: 빠르게 프로토타입을 만들고 테스트해 볼 수 있는 기능을 제공하여, 아이디어를 신속하게 검증할 수 있습니다.\n\n3. **커스터마이징**: 사용자 요구에 맞춰 모델을 커스터마이즈할 수 있는 옵션이 있어, 특정 업무나 도메인에 적합한 솔

In [23]:
print(results[0])
print(results[1])
print(results[2])

LangGraph은 언어와 관련된 다양한 데이터를 시각적으로 나타내고 분석할 수 있는 도구입니다. 더 쉽게 설명하자면, 텍스트와 단어의 관계를 그래프로 표현하여 언어의 구조와 의미를 이해하는 데 도움을 주는 플랫폼입니다.

1. **언어 데이터 시각화**: LangGraph은 단어, 문장, 또는 문서 간의 관계를 그래프 형태로 나타냅니다. 이를 통해 사용자는 특정 단어가 다른 단어와 어떻게 연결되어 있는지 쉽게 파악할 수 있습니다.

2. **관계 분석**: 예를 들어, 단어의 동의어, 대칭어, 그리고 문맥에 따라 어떻게 쓰이는지를 분석하여 언어의 뉘앙스를 이해하게 합니다.

3. **사용자 친화적인 인터페이스**: 복잡한 NLP(자연어 처리) 기술 없이도 그래프를 통해 데이터를 탐색하고 언어의 패턴을 발견할 수 있는 직관적인 인터페이스를 제공합니다.

4. **교육 및 연구 도구**: 언어학, 문학 연구, 교육 등의 분야에서 유용하게 사용할 수 있습니다. 학습자가 언어 구조를 이해하는 데 큰 도움이 됩니다.

이렇게 LangGraph은 언어의 패턴을 시각적으로 쉽게 이해하고, 활용할 수 있도록 돕는 도구입니다.
LangSmith는 주로 자연어 처리(NLP)와 관련된 도구 또는 플랫폼으로, 사용자들이 다양한 언어 모델을 쉽게 구축하고 관리할 수 있도록 돕는 솔루션입니다. 이 플랫폼은 개발자나 데이터 과학자가 복잡한 알고리즘이나 수학적 모델링 없이도 언어 모델을 실험하고 적용할 수 있는 환경을 제공합니다.

LangSmith의 주요 기능은 다음과 같습니다:

1. **모델 훈련**: 사용자들이 데이터를 입력하여 자신만의 언어 모델을 훈련시킬 수 있도록 지원합니다.

2. **프로토타이핑**: 빠르게 프로토타입을 만들고 테스트해 볼 수 있는 기능을 제공하여, 아이디어를 신속하게 검증할 수 있습니다.

3. **커스터마이징**: 사용자 요구에 맞춰 모델을 커스터마이즈할 수 있는 옵션이 있어, 특정 업무나 도메인에 적합한 솔루션을 구현할 수 있습니다.

4. **사용

In [27]:
# 4-2. 매서드사용
stream = chain.stream({"input":"LangSmith"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과:
LangSmith는 AI 언어 모델을 개발하고 관리하는 데 도움을 주는 도구입니다. 이 플랫폼은 주로 코딩 및 언어 처리 작업을 보다 효율적으로 수행할 수 있도록 설계되었습니다. 

주요 기능으로는 다음과 같은 것들이 있습니다:

1. **모델 훈련**: 사용자는 쉽게 AI 모델을 훈련시키고, 자신의 데이터를 기반으로 맞춤형 모델을 만들 수 있습니다.
2. **테스트 및 평가**: AI 모델의 성능을 테스트하고 평가할 수 있는 도구를 제공합니다.
3. **협업 기능**: 팀원들과의 협업을 통해 여러 사람과 함께 모델을 개발하고 개선할 수 있습니다.

따라서, LangSmith은 개발자나 데이터 과학자들이 보다 간편하게 AI 모델을 다루고, 이를 통해 다양한 언어 처리 작업을 수행할 수 있도록 돕는 플랫폼입니다.


## 프롬프트 작성 요령
- 질문은 명확하고 구체적
- 모델이 문맥을 이해할 수 있도록 필요한 배경 정보를 제공
- 핵심 정보에 초점을 맞추고, 불필요한 정보는 배제, 프롬프트가 길어지면 모델이 덜 중요한 부분에 집중하거나 상당한 영향을 받는 문제가 발생 가능
- 열린 질문을 통해 모델이 자세하고 풍부한 답변을 제공하도록 유도
- 얻고자 하는 정보나 결과의 유형을 정확하게 정의
- 대화의 맥락에 적합한 언어와 문체를 선택

In [14]:
template_text = "안녕하세요 제 이름은 {name}이고 , 나이는 {age}살입니다"
prompt_template = PromptTemplate.from_template(template_text)
filled_prompt = prompt_template.format(name = '김보생', age = '39')
filled_prompt

'안녕하세요 제 이름은 김보생이고 , 나이는 39살입니다'

In [15]:
combined_prompt = (prompt_template + prompt_template.from_template("\n\n아버지를 아버지라 부를 수 없습니다.") + "\n\n{language}로 번역해 주세요")
combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요 제 이름은 {name}이고 , 나이는 {age}살입니다\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해 주세요')

In [16]:
combined_prompt.format(name = "홍길동", age = '30', language = '영어')

'안녕하세요 제 이름은 홍길동이고 , 나이는 30살입니다\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해 주세요'

In [17]:
model = init_chat_model("gpt-4o-mini", model_provider="openai")
output_parser = StrOutputParser()

chain = combined_prompt | model | output_parser
chain.invoke({"name" : "홍길동", "age" : 30, "language":"english"})

'Hello, my name is Hong Gil-dong, and I am 30 years old.\n\nI cannot call my father "father."'

In [19]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user","{user_input}"),
])

message = chat_prompt.format_messages(user_input="안드로메다는 무엇인가요?")
message

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안드로메다는 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [20]:
chain = chat_prompt | model | StrOutputParser()
chain.invoke({"user_input":"안드로메다는 무엇인가요?"})

'안드로메다는 우리 galaxy인 은하수와 가장 가까운 큰 나선 은하 중 하나로, M31이라는 이름으로도 알려져 있습니다. 안드로메다는 지구에서 약 250만 광년 떨어져 있으며, 북반구의 겨울철 밤하늘에서 쉽게 관찰할 수 있습니다. \n\n안드로메다는 약 1조 개의 별을 포함하고 있으며, 크기는 대략 220,000 광년입니다. 이 은하는 태양계와 같은 중원형 구조를 가지고 있으며, 여러 개의 위성 은하(예: 삼각형자리 은하, M33)를 갖고 있습니다.\n\n안드로메다는 결국 우리 은하와 충돌할 것으로 예상되며, 그 과정은 수십억 년 후에 일어날 것으로 보입니다. 이러한 이유로 안드로메다는 천문학자들에게 연구의 중요한 대상이 되고 있습니다.'

In [21]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문: {question}\n{answer}")

In [22]:
examples = [
    {
        "question": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?",
        "answer": "지구 대기의 약 78%를 차지하는 질소입니다."
    },
    {
        "question": "광합성에 필요한 주요 요소들은 무엇인가요?",
        "answer": "광합성에 필요한 주요 요소는 빛, 이산화탄소, 물입니다."
    },
    {
        "question": "피타고라스 정리를 설명해주세요.",
        "answer": "피타고라스 정리는 직각삼각형에서 빗변의 제곱이 다른 두 변의 제곱의 합과 같다는 것입니다."
    },
    {
        "question": "지구의 자전 주기는 얼마인가요?",
        "answer": "지구의 자전 주기는 약 24시간(정확히는 23시간 56분 4초)입니다."
    },
    {
        "question": "DNA의 기본 구조를 간단히 설명해주세요.",
        "answer": "DNA는 두 개의 폴리뉴클레오티드 사슬이 이중 나선 구조를 이루고 있습니다."
    },
    {
        "question": "원주율(π)의 정의는 무엇인가요?",
        "answer": "원주율(π)은 원의 지름에 대한 원의 둘레의 비율입니다."
    }
]

In [ ]:
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
]

# 예제 프롬프트 템플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 현대과학의 아버지이며 당대최고의 천재입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = final_prompt | model

result = chain.invoke({"input": "시간이라는 개념은 존재하는겁니까?"})
print(result.content)

시간은 물리학과 철학에서 중요한 개념으로, 우리가 사건의 순서를 이해하고 변화와 운동을 측정하는 데 사용됩니다. 물리학적으로는 상대성 이론에 따라 시간은 공간과 함께 4차원 시공간의 일부로 간주됩니다. 그러나 시간의 본질에 대한 철학적 논의는 여전히 활발히 진행되고 있으며, 시간의 존재와 그 성격에 대한 다양한 이론이 존재합니다. 따라서 시간은 실질적으로 존재하는 개념이지만, 그 본질에 대한 이해는 복잡하고 다면적입니다.


In [44]:
print(result.content)

시간은 물리학과 철학에서 중요한 개념으로, 우리가 사건의 순서를 이해하고 변화와 운동을 측정하는 데 사용됩니다. 물리학적으로는 상대성 이론에 따라 시간은 공간과 함께 4차원 시공간의 일부로 간주됩니다. 그러나 시간의 본질에 대한 철학적 논의는 여전히 활발히 진행되고 있으며, 시간의 존재와 그 성격에 대한 다양한 이론이 존재합니다. 따라서 시간은 실질적으로 존재하는 개념이지만, 그 본질에 대한 이해는 복잡하고 다면적입니다.


In [46]:
pip install langchain_community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 73.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 12.9/12.9 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.2.6 which is incompatible.
tf-nightly-intel 2.19.0.dev20241121 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [ ]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)
 
# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

USER_AGENT environment variable not set, consider setting it to identify your requests.


1
13291
총체적으로 어기고 있다면 규범 준수를 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초

In [48]:
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}, page_content='\n\n\n\n위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문으로 이동\n\n\n\n\n\n\n\n주 메뉴\n\n\n\n\n\n주 메뉴\n사이드바로 이동\n숨기기\n\n\n\n\t\t둘러보기\n\t\n\n\n대문최근 바뀜요즘 화제임의의 문서로\n\n\n\n\n\n\t\t사용자 모임\n\t\n\n\n사랑방사용자 모임관리 요청\n\n\n\n\n\n\t\t편집 안내\n\t\n\n\n소개도움말정책과 지침질문방\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n보이기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n기부\n\n계정 만들기\n\n로그인\n\n\n\n\n\n\n\n\n개인 도구\n\n\n\n\n\n기부 계정 만들기 로그인\n\n\n\n\n\n\t\t로그아웃한 편집자를 위한 문서 더 알아보기\n\n\n\n기여토론\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n목차\n사이드바로 이동\n숨기기\n\n\n\n\n처음 위치\n\n\n\n\n\n1\n최상위 정책\n\n\n\n\n\n\n\n\n2\n\'정책과 지침\'이란?\n\n\n\n\n\n\n\n\n3\n준수\n\n\n\n\n\n\n\n\n4\n집행\n\n\n\n\n\n\n\n\n5\n문서 내용\

In [51]:
print(docs[0].page_content)





위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전



























본문으로 이동







주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로





		사용자 모임
	


사랑방사용자 모임관리 요청





		편집 안내
	


소개도움말정책과 지침질문방



















검색











검색






















보이기
















기부

계정 만들기

로그인








개인 도구





기부 계정 만들기 로그인





		로그아웃한 편집자를 위한 문서 더 알아보기



기여토론




























목차
사이드바로 이동
숨기기




처음 위치





1
최상위 정책








2
'정책과 지침'이란?








3
준수








4
집행








5
문서 내용








6
정책과 지침은 백과사전의 일부가 아닙니다








7
채택 과정




채택 과정 하위섹션 토글하기





7.1
제안과 채택








7.2
내용 변경






7.2.1
실질적인 변경










7.3
격하










8
같이 보기








9
외부 링크


















목차 토글







위키백과:정책과 지침



112개 언어




Afrikaansአማርኛअंगिकाالعربيةالدارجةمصرىঅসমীয়াАварAzərbaycancaتۆرکجهБашҡортсаBoarischБеларускаяБеларуская (тарашкевіца)БългарскиभोजपुरीBanjarবাংলাBrezhonegBosanskiCatalàНохчийнکوردیČeštinaCymraegDanskDeutschΕλληνικάEnglishEsperantoEspañolEuskaraفارسیSuomiFrançaisGaei

In [52]:
# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])

19
page_content='제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.' metadata={'source': '

In [53]:
# page_content 속성
splits[10].page_content

'제안과 채택\n\xa0백:아님 §\xa0관료주의  문서를 참고하십시오. 단축백:제안\n제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.\n\'제안\'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.\n좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.\n정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.'

In [56]:
pip install Chromadb

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/19.0 MB ? eta -:--:--
   ---------------------------------------  18.9/19.0 MB 99.5 MB/s eta 0:00:01
   ---------------------------------------- 19.0/19.0 MB 86.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 53.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 12.7/12.7 MB 79.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 70.1 MB/s eta 0:00:00
 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.


In [57]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
print(len(docs))
print(docs[0].page_content)

4
격하
특정 정책이나 지침이 편집 관행이나 공동체 규범이 바뀌며 쓸모없어질 수 있고, 다른 문서가 개선되어 내용이 중복될 수 있으며, 불필요한 내용이 증식할 수도 있습니다. 이 경우 편집자들은 정책을 지침으로 격하하거나, 정책 또는 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 격하할 것을 제안할 수 있습니다. 
격하 과정은 채택 과정과 비슷합니다. 일반적으로 토론 문서 내 논의가 시작되고 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 논의가 충분히 이루어진 후, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되었는지 판단해야 합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 정책/지침임을 알립니다.
소수의 공동체 인원만 지지하는 수필, 정보문 및 기타 비공식 문서는 일반적으로 주된 작성자의 사용자 이름공간으로 이동합니다. 이러한 논의는 일반적으로 해당 문서의 토론란에서 이루어지며, 간혹 위키백과:의견 요청을 통해 처리되기도 합니다.

같이 보기
위키백과:위키백과의 정책과 지침 목록
위키백과:의견 요청
수필

위키백과:제품, 절차, 정책
위키백과:위키백과 공동체의 기대와 규범
기타 링크
